In [1]:
%pip install -r requirements.txt

You should consider upgrading via the '/Users/aryanlangeh/algo-trading-bot/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [142]:
import pandas_datareader.data as web
import pandas as pd
from datetime import datetime
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np
import backtrader as bt
import datetime

In [133]:
eur_usd = yf.download('EURUSD=X', start='2022-07-01', end='2022-08-01')
eur_usd = eur_usd.dropna()
eur_usd = eur_usd.drop(columns=['Open', 'High', 'Low', 'Adj Close', 'Volume'])
eur_usd = eur_usd.rename(columns={'Close': 'close'})
eur_usd_df = pd.DataFrame(eur_usd)
eur_usd_bt = bt.feeds.PandasData(dataname=eur_usd_df)

eur_gbp = yf.download('EURGBP=X', start='2022-07-01', end='2022-08-01')
eur_gbp = eur_gbp.dropna()
eur_gbp = eur_gbp.drop(columns=['Open', 'High', 'Low', 'Adj Close', 'Volume'])
eur_gbp = eur_gbp.rename(columns={'Close': 'close'})
eur_gbp_df = pd.DataFrame(eur_gbp)
eur_gbp_bt = bt.feeds.PandasData(dataname=eur_gbp_df)

gbp_usd = yf.download('GBPUSD=X', start='2022-07-01', end='2022-08-01')
gbp_usd = gbp_usd.dropna()
gbp_usd = gbp_usd.drop(columns=['Open', 'High', 'Low', 'Adj Close', 'Volume'])
gbp_usd = gbp_usd.rename(columns={'Close': 'close'})
gbp_usd_df = pd.DataFrame(gbp_usd)
gbp_usd_bt = bt.feeds.PandasData(dataname=gbp_usd_df)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [39]:
gbp_usd_df.tail()

,close
Date,
2022-07-25,1.198696
2022-07-26,1.205516
2022-07-27,1.203688
2022-07-28,1.216308
2022-07-29,1.217345


In [40]:
eur_usd_df.tail()

,close
Date,
2022-07-25,1.020023
2022-07-26,1.022495
2022-07-27,1.012956
2022-07-28,1.020929
2022-07-29,1.019233


In [41]:
eur_gbp_df.tail()

,close
Date,
2022-07-25,0.85088
2022-07-26,0.84800
2022-07-27,0.84149
2022-07-28,0.83905
2022-07-29,0.83720


In [8]:
# Generate synthetic Forex data
np.random.seed(42)
n = 500  # Number of data points

date_rng = pd.date_range(start='2020-01-01', end='2020-01-30', freq='H')
eurusd_data = pd.DataFrame(date_rng, columns=['date'])
eurusd_data['close'] = 1.18 + np.random.normal(0, 0.001, size=(len(date_rng)))

eurgbp_data = pd.DataFrame(date_rng, columns=['date'])
eurgbp_data['close'] = 0.86 + np.random.normal(0, 0.001, size=(len(date_rng)))

gbpusd_data = pd.DataFrame(date_rng, columns=['date'])
gbpusd_data['close'] = 1.38 + np.random.normal(0, 0.001, size=(len(date_rng)))

# Convert to backtrader data feed
eurusd_bt = bt.feeds.PandasData(dataname=eurusd_data, datetime='date', close='close')
eurgbp_bt = bt.feeds.PandasData(dataname=eurgbp_data, datetime='date', close='close')
gbpusd_bt = bt.feeds.PandasData(dataname=gbpusd_data, datetime='date', close='close')


In [54]:
%pip install networkx

     |████████████████████████████████| 2.1 MB 4.6 MB/s eta 0:00:016 MB/s eta 0:00:01
You should consider upgrading via the '/Users/aryanlangeh/algo-trading-bot/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [55]:
import networkx as nx
import math

In [112]:
class TriangularArbitrage(bt.Strategy):
    params = (
        ('transaction_cost', 0.005),
    )

    def __init__(self):
        self.eurusd = self.datas[0]
        self.eurgbp = self.datas[1]
        self.gbpusd = self.datas[2]
        self.graph = nx.DiGraph()
        self.graph.add_nodes_from(['EUR', 'USD', 'GBP'])
        self.graph.add_edge('EUR', 'USD', weight=0)
        self.graph.add_edge('USD', 'EUR', weight=0)
        self.graph.add_edge('EUR', 'GBP', weight=0)
        self.graph.add_edge('GBP', 'EUR', weight=0)
        self.graph.add_edge('GBP', 'USD', weight=0)
        self.graph.add_edge('USD', 'GBP', weight=0)


    def next(self):
        portfolio_value = self.broker.getvalue()
        stake = portfolio_value / 3  # dividing by 3 as we are trading with 3 pairs

        print(f"Current Portfolio Value: {self.broker.getvalue()}")
        # print(f"Position size for EURUSD: {self.broker.getposition(self.eurusd).size}")
        # print(f"Position size for EURGBP: {self.broker.getposition(self.eurgbp).size}")
        # print(f"Position size for GBPUSD: {self.broker.getposition(self.gbpusd).size}")
        # Update the graph with the latest exchange rates
        self.graph['EUR']['USD']['weight'] = -math.log(self.eurusd.close[0])
        self.graph['USD']['EUR']['weight'] = -math.log(1 / self.eurusd.close[0])
        self.graph['EUR']['GBP']['weight'] = -math.log(self.eurgbp.close[0])
        self.graph['GBP']['EUR']['weight'] = -math.log(1 / self.eurgbp.close[0])
        self.graph['GBP']['USD']['weight'] = -math.log(self.gbpusd.close[0])
        self.graph['USD']['GBP']['weight'] = -math.log(1 / self.gbpusd.close[0])

        # Check for arbitrage opportunities by finding negative cycles
        # This is a naive implementation just for demonstration. More efficient algorithms are available.
        for cycle in nx.simple_cycles(self.graph):
            if len(cycle) == 3:  # Triangular arbitrage
                weight = sum(self.graph[cycle[i]][cycle[i + 1]]['weight'] for i in range(len(cycle) - 1))
                weight += self.graph[cycle[-1]][cycle[0]]['weight']

                if weight < -math.log(1 - self.params.transaction_cost):
                    self.execute_trade(cycle)

    def execute_trade(self, cycle):
        stake = self.broker.getvalue() / 3
        print(f"Stake before trade: {stake}")  # Debugging line

        for i in range(len(cycle) - 1):
            self.execute_order(cycle[i], cycle[i + 1], stake)
            print(f"Cash: {self.broker.get_cash()}")  # Debugging line
            print(f"Portfolio value after {cycle[i]} to {cycle[i + 1]} trade: {self.broker.getvalue()}")  # Debugging line
            if math.isnan(self.broker.getvalue()):
                print(f"Something went wrong after {cycle[i]} to {cycle[i + 1]} trade")
                break

        self.execute_order(cycle[-1], cycle[0], stake)
        print(f"Cash: {self.broker.get_cash()}")  # Debugging line
        print(f"Portfolio value after {cycle[-1]} to {cycle[0]} trade: {self.broker.getvalue()}")  # Debugging line



    def execute_order(self, from_currency, to_currency, stake_fraction):
        
        if from_currency == 'EUR' and to_currency == 'USD':
            size = stake_fraction / self.eurusd.close[0]
            #print(f"Size: {size}, Close Price: {self.eurusd.close[0]}")  # Debugging line
            self.buy(data=self.eurusd, size=size)

        elif from_currency == 'USD' and to_currency == 'EUR':
            size = stake_fraction * self.eurusd.close[0]
            #print(f"Size: {size}, Close Price: {self.eurusd.close[0]}")  # Debugging line
            self.sell(data=self.eurusd, size=size)

        elif from_currency == 'EUR' and to_currency == 'GBP':
            size = stake_fraction / self.eurgbp.close[0]
            #print(f"Size: {size}, Close Price: {self.eurgbp.close[0]}")  # Debugging line
            self.buy(data=self.eurgbp, size=size)

        elif from_currency == 'GBP' and to_currency == 'EUR':
            size = stake_fraction * self.eurgbp.close[0]
            #print(f"Size: {size}, Close Price: {self.eurgbp.close[0]}")  # Debugging line
            self.sell(data=self.eurgbp, size=size)

        elif from_currency == 'GBP' and to_currency == 'USD':
            size = stake_fraction / self.gbpusd.close[0]
            #print(f"Size: {size}, Close Price: {self.gbpusd.close[0]}")  # Debugging line
            self.buy(data=self.gbpusd, size=size)

        elif from_currency == 'USD' and to_currency == 'GBP':
            size = stake_fraction * self.gbpusd.close[0]
            #print(f"Size: {size}, Close Price: {self.gbpusd.close[0]}")  # Debugging line
            self.sell(data=self.gbpusd, size=size)

        #print(f"Executed trade from {from_currency} to {to_currency}")



In [190]:
class TriangularArbitrage1(bt.Strategy): # fix end after trade value in usd
    
    def __init__(self):
        self.data_eur_usd = self.datas[0].close  # Data for EUR/USD
        self.data_eur_gbp = self.datas[1].close  # Data for EUR/GBP
        self.data_gbp_usd = self.datas[2].close  # Data for GBP/USD

        # Assume we start with 1000 USD
        self.start_cash = 1000
        self.cash = self.start_cash

    def next(self):
        eur_usd = self.data_eur_usd[0]
        gbp_usd = self.data_gbp_usd[0]
        eur_gbp = self.data_eur_gbp[0]
        
        date_num = self.datas[0].datetime[0]  # for EUR/USD
        date = bt.num2date(date_num).date()

        paths_end = [
            (self.cash / eur_usd / eur_gbp) * gbp_usd,
            (self.cash / gbp_usd * eur_gbp) / eur_usd,
            (self.cash / eur_usd) * gbp_usd / eur_gbp,
            (self.cash / gbp_usd) / eur_usd * eur_gbp,
            (self.cash * eur_gbp / gbp_usd) * eur_usd,
            (self.cash * eur_gbp) * eur_usd / gbp_usd,
        ]

        best_path_end = max(paths_end)

        if best_path_end > self.cash:
            idx = paths_end.index(best_path_end)
            
            print(f"Date: {date}, Arbitrage opportunity found in Path {idx}!")
            
            if idx == 0:
                intermediate1 = self.cash / eur_usd
                intermediate2 = intermediate1 * eur_gbp
                print(f"Intermediate (USD to EUR): {intermediate1} EUR")
                print(f"Intermediate (EUR to GBP): {intermediate2} GBP")
            
            elif idx == 1:
                intermediate1 = self.cash / gbp_usd
                intermediate2 = intermediate1 / eur_gbp
                print(f"Intermediate (USD to GBP): {intermediate1} GBP")
                print(f"Intermediate (GBP to EUR): {intermediate2} EUR")

            elif idx == 2:
                intermediate1 = self.cash / eur_usd
                intermediate2 = intermediate1 * gbp_usd / eur_gbp
                print(f"Intermediate (USD to EUR): {intermediate1} EUR")
                print(f"Intermediate (EUR to GBP): {intermediate2} GBP")

            elif idx == 3:
                intermediate1 = self.cash / gbp_usd
                intermediate2 = intermediate1 / eur_usd * eur_gbp
                print(f"Intermediate (USD to GBP): {intermediate1} GBP")
                print(f"Intermediate (GBP to EUR): {intermediate2} EUR")

            elif idx == 4:
                intermediate1 = self.cash * eur_gbp
                intermediate2 = intermediate1 / gbp_usd
                print(f"Intermediate (USD to EUR): {intermediate1} EUR")
                print(f"Intermediate (EUR to GBP): {intermediate2} GBP")

            elif idx == 5:
                intermediate1 = self.cash * eur_gbp
                intermediate2 = intermediate1 * eur_usd / gbp_usd
                print(f"Intermediate (USD to EUR): {intermediate1} EUR")
                print(f"Intermediate (EUR to GBP): {intermediate2} GBP")

            print(f"Execute trades. Start: {self.cash} USD, End: {intermediate2} USD")
            
            self.cash = intermediate2

In [191]:
cerebro = bt.Cerebro()

# Add data feeds to Cerebro
cerebro.adddata(eur_usd_bt, name="EURUSD")
cerebro.adddata(eur_gbp_bt, name="EURGBP")
cerebro.adddata(gbp_usd_bt, name="GBPUSD")


# cerebro.broker.setcash(120)
# Add strategy
cerebro.addstrategy(TriangularArbitrage1)

# Add analyzers
# cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
# cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
# cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')

#cerebro.broker.setcommission(commission=0.00005)

# Run backtest
    # Run backtest
strategy = cerebro.run()[0]

    # Print final value of cash
print(f"Final Cash Value: {strategy.cash} USD")


# Print results
# print(f"Sharpe Ratio: {results[0].analyzers.sharpe.get_analysis()['sharperatio']}")
# print(f"Max Drawdown: {results[0].analyzers.drawdown.get_analysis()['max']['drawdown']}")
# print(f"Total Returns: {results[0].analyzers.returns.get_analysis()['rtot']}")

# # Plot the strategy
# cerebro.plot()


Date: 2022-07-01, Arbitrage opportunity found in Path 0!
Intermediate (USD to EUR): 954.4099749320567 EUR
Intermediate (EUR to GBP): 822.26238842558 GBP
Execute trades. Start: 1000 USD, End: 822.26238842558 USD
Date: 2022-07-04, Arbitrage opportunity found in Path 0!
Intermediate (USD to EUR): 788.064484577313 EUR
Intermediate (EUR to GBP): 679.4692157380557 GBP
Execute trades. Start: 822.26238842558 USD, End: 679.4692157380557 USD
Date: 2022-07-05, Arbitrage opportunity found in Path 2!
Intermediate (USD to EUR): 651.2576279950448 EUR
Intermediate (EUR to GBP): 916.1527359328861 GBP
Execute trades. Start: 679.4692157380557 USD, End: 916.1527359328861 USD
Date: 2022-07-06, Arbitrage opportunity found in Path 0!
Intermediate (USD to EUR): 893.1389335517198 EUR
Intermediate (EUR to GBP): 766.6079509884705 GBP
Execute trades. Start: 916.1527359328861 USD, End: 766.6079509884705 USD
Date: 2022-07-07, Arbitrage opportunity found in Path 0!
Intermediate (USD to EUR): 752.5790391488865 EUR
In

In [164]:
eur_usd.to_csv('EURUSD.csv')
eur_gbp.to_csv('EURGBP.csv')
gbp_usd.to_csv('GBPUSD.csv')